In [ ]:
!pip install -Uqq datasets
!pip install -Uqq scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 92.8 MB/s eta 0:00:00


## Load dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("carblacac/twitter-sentiment-analysis")
print(dataset)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

...

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'feeling'],
        num_rows: 119988
    })
    validation: Dataset({
        features: ['text', 'feeling'],
        num_rows: 29997
    })
    test: Dataset({
        features: ['text', 'feeling'],
        num_rows: 61998
    })
})


## Split dataset

In [ ]:
train_data = dataset["train"]
test_data = dataset["test"]

# sample data point
sample = train_data[0]
print("Sample:", sample)

Sample: {'text': '@fa6ami86 so happy that salman won.  btw the 14sec clip is truely a teaser', 'feeling': 0}


## Transform data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

train_text = [item['text'] for item in train_data]
train_labels = [item['feeling'] for item in train_data]

test_text = [item['text'] for item in test_data]
test_labels = [item['feeling'] for item in test_data]

# Vectorize
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(train_text)
X_test_counts = count_vectorizer.transform(test_text)

# TF-IDF representation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)


## Train data using Naive Bayes classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train_tfidf, train_labels)
pred_labels = mnb.predict(X_test_tfidf)


## Evaluate Model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report


print("Naive Bayes Classifier:\n")

print(" Test Set Accuracy:", accuracy_score(test_labels, pred_labels))
print(" Classification Report:")
print(classification_report(test_labels, pred_labels))

Naive Bayes Classifier:

 Test Set Accuracy: 0.759895480499371
 Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.83      0.78     30969
           1       0.80      0.69      0.74     31029

    accuracy                           0.76     61998
   macro avg       0.77      0.76      0.76     61998
weighted avg       0.77      0.76      0.76     61998

